In [2]:
!pip install autogluon

In [3]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor

directory = '/content/dataset/'
label = 'isFraud'
save_path = '/content/model/'

train_identity = pd.read_csv(directory + 'train_identity.csv')
train_transaction = pd.read_csv(directory + 'train_transaction.csv')

In [4]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
train_data = train_data.sample(frac=0.1, random_state=42)

In [5]:
predictor = TabularPredictor(label=label, path=save_path, verbosity=3).fit(
    train_data, presets='medium_quality', num_bag_folds=2, time_limit=3600
)

results = predictor.fit_summary()

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          0
Memory Avail:       10.71 GB / 12.67 GB (84.5%)
Disk Space Avail:   64.39 GB / 107.72 GB (59.8%)
Presets specified: ['medium_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': False, 'num_bag_folds': 2}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.1111111111111111,
             'memory_safe_fits': True,
             'n_folds': 

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L2     0.9760    accuracy      10.323981  290.781969                0.001956           0.522552            2       True         14
1     NeuralNetTorch_BAG_L1     0.9756    accuracy       9.084150  232.067105                9.084150         232.067105            1       True         12
2     ExtraTreesEntr_BAG_L1     0.9752    accuracy       0.984262    4.086348                0.984262           4.086348            1       True          9
3   RandomForestGini_BAG_L1     0.9750    accuracy       1.088833    5.094760                1.088833           5.094760            1       True          5
4         LightGBMXT_BAG_L1     0.9748    accuracy       1.237875   58.192313                1.237875          58.192313            1       True          3
5 

In [6]:
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')
test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

y_predproba = predictor.predict_proba(test_data)
y_predproba.head(5)  # some example predicted fraud-probabilities

Loading: /content/model/models/LightGBMXT_BAG_L1/model.pkl
/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
Loading: /content/model/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: /content/model/models/WeightedEnsemble_L2/model.pkl


,0,1
0,0.999403,0.000597
1,0.999215,0.000785
2,0.998718,0.001282
3,0.999470,0.000530
4,0.999175,0.000825


In [7]:
predictor.positive_class

1

In [8]:
predictor.class_labels

[0, 1]

In [9]:
y_predproba = predictor.predict_proba(test_data, as_multiclass=False)

Loading: /content/model/models/LightGBMXT_BAG_L1/model.pkl
Loading: /content/model/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: /content/model/models/WeightedEnsemble_L2/model.pkl


In [10]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)